In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 30, 5

In [ ]:
import numpy   as np
import pandas  as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.utils import np_utils
from sklearn.metrics import confusion_matrix,classification_report

In [ ]:
df = pd.read_csv("../input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv")
df.head()

# EDA

In [ ]:
columns = ['age', 'anaemia', 'diabetes',
       'ejection_fraction', 'high_blood_pressure',
       'serum_creatinine', 'serum_sodium', 'sex', 'smoking',
       'DEATH_EVENT']

In [ ]:
def plot_data(name):
    result = df[[name]].value_counts()
    result.plot(kind="bar")

In [ ]:
for item in columns:
    plot_data(item)
    plt.legend()
    plt.figure()

In [ ]:
rel_with_target = df.corr()[['DEATH_EVENT']].sort_values(['DEATH_EVENT'],ascending=True)
rel_with_target.plot(kind="bar")

In [ ]:
rel_with_target.plot()

In [ ]:
rel_with_target

In [ ]:
sns.heatmap(df.corr())

In [ ]:
feature_matrix = df.drop("DEATH_EVENT",axis=1)
target         = df[['DEATH_EVENT']]

In [ ]:
new_target = np_utils.to_categorical(target)

In [ ]:
normalized_fm=(feature_matrix-feature_matrix.min())/(feature_matrix.max()-feature_matrix.min())

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(normalized_fm,new_target,test_size=.2)

In [ ]:
n_col = X_train.shape[1]

In [ ]:
def build_model(n_col):
    model = Sequential()
    model.add(Dense(32, input_dim=n_col, activation='relu'))
    model.add(Dropout(.1))
    model.add(Dense(32, input_dim=n_col, activation='relu'))
    model.add(Dropout(.1))
    model.add(Dense(64, input_dim=n_col, activation='relu'))
    model.add(Dropout(.1))
    model.add(Dense(64, input_dim=n_col, activation='relu'))
    model.add(Dropout(.1))
    model.add(Dense(64, input_dim=n_col, activation='relu'))
    model.add(Dropout(.1))
    model.add(Dense(64, input_dim=n_col, activation='relu'))
    model.add(Dropout(.1))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    # compile the keras model
    model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])
    return model

In [ ]:
model= build_model(n_col)

In [ ]:
model.summary()

In [ ]:
history = model.fit(X_train,y_train,epochs=100,validation_data=(X_test, y_test))

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
predicted_value = []
for item in model.predict(X_test):
    predicted_value.append(np.argmax(item))
print(predicted_value)    
    
    

In [ ]:
actual_result = []
for item in y_test:
    actual_result.append(np.argmax(item))

In [ ]:
pd.DataFrame(predicted_value).value_counts().plot(kind="bar")

In [ ]:
pd.DataFrame(actual_result).value_counts().plot(kind="bar")

In [ ]:
loss,acc = model.evaluate(X_test,y_test)

In [ ]:
print("LOSS OF THE MODEL     : {}".format(loss))
print("ACCURACY OF THE MODEL : {}".format(acc))

In [ ]:
report = classification_report(y_pred=predicted_value,y_true=actual_result)

In [ ]:
print(report)

In [ ]:
cm = confusion_matrix(predicted_value,actual_result)

In [ ]:
sns.heatmap(cm,annot=True)